# 1 Author

**Student Name**: Zhong Zhenghan 

**Student ID**:  210982480

**Github**:  https://github.com/buptxinghan/CBU5201_miniproject_Zhenghan


# 2 Problem formulation
- 微笑检测：确定图片中的人是否在微笑。
- 3D 头部姿态估计：预测头部的3D方向。

# 3 Machine Learning pipeline

- 输入：genki4k 数据集中的一张图片。
- 转换：使用 Haar 级联或 CNN 进行人脸检测和裁剪，数据标准化和增强，使用 CNN 提取特征。
- 模型：微笑检测的分类模型和3D头部姿态的回归模型。
- 输出：微笑/非微笑的二元标签和3D头部姿态标签。

# 4 Transformation stage

- 图像大小调整：所有图像都被调整为统一的大小,数据标准化（像素值缩放）和增强（旋转，翻转等）。
- 使用预训练模型（如 Haar 级联或 CNN）进行面部检测和裁剪。
- 颜色空间转换：如果需要，图像颜色空间从 RGB 转换为灰度。
- 特征工程：提取与微笑和头部姿态相关的特征,使用大型数据集预训练的 CNN（例如 VGG16）提取特征。

# 5 Modelling

- 微笑检测：二元分类模型，例如逻辑回归、SVM 或简单的 CNN。
- 3D 头部姿态估计：回归模型，可以是多层感知器或 CNN。

# 6 Methodology

- 数据划分：训练/验证/测试数据集比例为70%/15%/15%。
- 交叉验证：使用交叉验证来评估模型的泛化能力。
- 性能指标：微笑检测的准确度和混淆矩阵；姿态估计的均方误差。

# 7 Dataset

- 预处理 genki4k 数据集，可视化一些样本。
- 执行任何必要的清理或增强。

# 8 Results

- 展示模型的性能：包括训练过程中的损失和准确度变化图、验证集和测试集上的最终结果。
- 分析结果：讲述模型表现好的地方，以及可能的原因。同时，如果模型某些方面表现不佳，提出可能的原因和解决办法。

# 9 Conclusions

- 概述您的项目和实验结果。
- 讨论项目的成功点和存在的局限。
- 提出改进模型或实验的可能方法。
- 反思整个项目过程，包括学到了什么，遇到了哪些挑战，以及如何克服这些挑战。

在这个项目中，我们成功地实现了一个用于识别人脸微笑的模型。通过使用卷积神经网络，我们在测试集上达到了 XX% 的准确率。尽管结果是令人满意的，但我们发现模型在处理不同光照条件下的图像时存在一定的局限性。未来的工作可以在数据增强和模型鲁棒性方面进行更多的探索。在这个过程中，我们学习了如何处理图像数据、如何选择合适的模型结构以及如何调整模型参数来优化性能。

# 10 Training process

1. 导入库

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

2. 数据加载与预处理

In [2]:
# 图片和标签的存储路径
image_folder_path = 'D:/Documents/Junior1/Machine Learning/LAB/Mini Project/genki4k/files'
labels_file_path = 'D:/Documents/Junior1/Machine Learning/LAB/Mini Project/genki4k/labels.txt'

# 读取标签文件
with open(labels_file_path, 'r') as file:
    labels = file.read().splitlines()

# 确保标签数量与图像文件数量匹配
assert len(labels) == len(os.listdir(image_folder_path))

# 获取所有图像文件的路径
image_paths = [os.path.join(image_folder_path, fname) for fname in sorted(os.listdir(image_folder_path)) if fname.endswith('.jpg')]

# 创建 DataFrame
data = pd.DataFrame({
    'file_path': image_paths,
    'label': labels
})

def preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Could not read image {image_path}. It's either missing or not readable.")
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize the image
    return image

# Apply data preprocessing
# Note: If the dataset is large, this operation might consume a large amount of memory and time.
data['image'] = data['file_path'].apply(preprocess_image)

3. 数据分割

In [4]:
# 假设标签在 'smile_label' 列（对于微笑检测）
X_train, X_test, y_train, y_test = train_test_split(data['image'], data['label'], test_size=0.15, random_state=42)

4. 模型构建

In [5]:
# 构建模型
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

5. 模型训练

In [7]:
# 模型训练
history = model.fit(X_train, y_train, epochs=10, validation_split=0.15)

TypeError: Can't convert object to 'str' for 'filename'

6. 模型评估

In [ ]:
# 模型评估
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

# 计算准确度
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# 混淆矩阵
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

7. 结果可视化

In [ ]:
# 准确度和损失随时间变化的图表
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy over epochs')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss over epochs')

plt.show()